In [6]:
import pm4py
import pandas as pd
import re

In [35]:
df = pd.read_csv('BPIC15_1.csv')

/tmp/ipykernel_176/395401240.py:1: DtypeWarning: Columns (29) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('BPIC15_1.csv')


In [39]:
dataframe = pm4py.format_dataframe(dataframe)

/tmp/ipykernel_176/1384693449.py:1: DeprecatedWarning: format_dataframe is deprecated as of 2.3.0 and will be removed in 3.0.0. the format_dataframe function does not need application anymore.
  dataframe = pm4py.format_dataframe(dataframe)


In [41]:
log = pm4py.convert_to_event_log(dataframe)

/opt/conda/lib/python3.9/site-packages/pm4py/utils.py:486: UserWarning: the EventLog class has been deprecated and will be removed in a future release.
  warnings.warn("the EventLog class has been deprecated and will be removed in a future release.")


In [ ]:
net, im, fm = pm4py.discover_petri_net_inductive(dataframe)